In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in data from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"), inferSchema=True, sep=r'\t', dateFormat="yyyy-MM-dd HH:mm:ss")
df.show(10)

In [ ]:
df.printSchema()

In [ ]:
print(df.count())

In [ ]:
dropna_df=df.dropna(how = "any")
dropna_df.show(truncate = False)
dropna_df.count()

In [ ]:
dropduplicates_df=dropna_df.dropna(how = "any")
dropduplicates_df.show(truncate = False)
dropduplicates_df.count()

In [ ]:
dropduplicates_df.printSchema()

In [ ]:
clean = dropduplicates_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])

clean.show(5)




In [ ]:
clean.printSchema()

In [ ]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col, unix_timestamp, to_date

# changedate = clean.withColumn("review_date", to_date("review_date", "D M y"))

# changedate.show()


review_id_table = clean.withColumn('review_date', to_date(unix_timestamp(col('review_date'), 'yyyy-MM-dd').cast("timestamp")))
review_id_table.show()
# review_id_table.printSchema()

In [ ]:
review_id_table.printSchema()

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://postgresdb.cdgek8t95yas.ap-southeast-2.rds.amazonaws.com:5432/alexa_db"
config = {"user":"master", 
          "password": "HJCAWSpostgres", 
          "driver":"org.postgresql.Driver"}

In [ ]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)